In [1]:
import Pkg
Pkg.add("Pkg")
Pkg.add("Dates")

guypath = "/Users/guyfarmer/ambulance_connect/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/"
michaelpath= "/home/mhilborn/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy"

currentpath = michaelpath

cd(currentpath)
Pkg.activate(".")

using DataFrames, JLD, Distributions, CSV, Random, Plots,JSON
import DataStructures: PriorityQueue, enqueue!, dequeue!

include(".//src//model.jl")

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...
No Changes to `~/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/test/Project.toml`
No Changes to `~/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/test/Manifest.toml`
  Resolving package versions...
No Changes to `~/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/test/Project.toml`
No Changes to `~/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/test/Manifest.toml`
 Activating environment at `~/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Project.toml`
  Resolving package versions...
No Changes to `~/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Project.toml`
No Changes to `~/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Manifest.toml`
  Resolving package versions...
No Changes to `~/Desktop/UT-Spring2021

reassign_ambulances! (generic function with 2 methods)

In [3]:
#=
Author : Ng Yeesian
Modified : Guy Farmer / Zander Tedjo / Will Worthington / Michael Hilborn
generates the closest dispatch model
=#
mutable struct ClosestDispatch <: DispatchModel
    drivetime::DataFrame
    candidates::Vector{Vector{Int}}
    available::Vector{Int}
    assignment::Vector{Int} # which location the ambulance is assigned to
    ambulances::Vector{Vector{Int}} # list of ambulances assigned to each location
    status::Vector{Symbol} # the current status of the ambulance
        # possible statuses: :available, :responding, :atscene, :conveying, :returning
    fromtime::Vector{Int} # the time it started the new status
    hospital::Vector{Int}
end

function ClosestDispatch(p::DeploymentProblem, drivetime::DataFrame,distribution::Vector{Int})
    candidates = Any[]
    I = 1:p.nlocations
    for region in 1:p.nregions
        push!(candidates, I[vec(p.coverage[region,:])])
    end
    #println("creating the dispatch problem :$distribution")

    assignment = zeros(Int, p.nambulances)
    ambulances = [Int[] for i in 1:p.nlocations]
    k = 1
    for i in eachindex(distribution), j in 1:distribution[i]
        assignment[k] = i
        push!(ambulances[i], k)
        k += 1
    end
    status = fill(:available, p.nambulances)
    fromtime = zeros(Int, p.nambulances)
    hospital = zeros(Int, p.nambulances)
    
    ClosestDispatch(drivetime, candidates, distribution, assignment, ambulances, status, fromtime, hospital)

end

function ClosestDispatch(p::DeploymentProblem, problem::DispatchProblem)
    stn_names = [Symbol("stn$(i)_min") for i in 1:size(p.coverage,2)]
    ClosestDispatch(p, problem.emergency_calls[:, stn_names], problem.available)
end

function update_ambulances!(model::ClosestDispatch, i::Int, delta::Int)
    model.available[i] += delta
    @assert model.available[i] >= 0
end

function closest_available(dispatch::ClosestDispatch, id::Int, problem::DispatchProblem)
    location = 0
    min_time = Inf
    for i in dispatch.candidates[problem.emergency_calls[id, :neighborhood]]
        if problem.available[i] > 0 && dispatch.drivetime[id, i] < min_time
            #println("apparently this station is available : $i and here is the amount of ambulances avalable: $(problem.available[i])")
            location = i
            min_time = dispatch.drivetime[id, i]
        end
    end
    location
end

function respond_to!(dispatch::ClosestDispatch, i::Int, t::Int)
    @assert length(dispatch.ambulances[i]) > 0 "$i: $(dispatch.ambulances[i])"
    amb = popfirst!(dispatch.ambulances[i])
    # @assert redeploy.hospital[amb] == 0
    @assert amb != 0
    @assert dispatch.status[amb] == :available "$amb: $(dispatch.status[amb])"
    dispatch.status[amb] = :responding
    #dispatch.fromtime[amb] = t
    amb
end

function arriveatscene!(dispatch::ClosestDispatch, amb::Int, t::Int)
    @assert dispatch.status[amb] == :responding "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] == 0
    dispatch.status[amb] = :atscene
    dispatch.fromtime[amb] = t
end

function arriveathospital!(dispatch::DispatchModel, amb::Int, hosp::Int, t::Int)
    @assert dispatch.status[amb] == :atscene "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] != 0
    dispatch.status[amb] = :atHospital
    dispatch.fromtime[amb] = t
    dispatch.hospital[amb] = hosp
end

function returning_to!(dispatch::DispatchModel, amb::Int, t::Int)
    @assert dispatch.status[amb] == :atHospital "$amb: $(dispatch.status[amb])"
    @assert dispatch.hospital[amb] != 0
    dispatch.status[amb] = :returning
    dispatch.fromtime[amb] = t
    dispatch.assignment[amb]
end

returning_to! (generic function with 2 methods)

In [4]:
turnaround = Distributions.LogNormal(3.65, 0.3)
lambda = 0
local_path = ""
model_name = "Stochastic"
namb = 40
ncalls = 1000
#using AmbulanceDeployment

pwd()

cd(currentpath * "/test/")
pwd()

#using AmbulanceDeployment
using Dates

#include("..//src//dispatch/closestdispatch.jl")
include("..//src//problem.jl")
include("..//src//evaluate.jl")
#include("..//src//simulate.jl")

hospitals = CSV.File(string(local_path,"../test/austin-data/hospitals.csv")) |> DataFrame
stations = CSV.File(string(local_path,"../test/austin-data/stations.csv")) |> DataFrame
hourly_calls = CSV.File("../test/austin-data/Full_WeekdayCalls.csv") |> DataFrame
adjacent_nbhd = CSV.File("../test/austin-data/adjacent_nbhd.csv") |> DataFrame
coverage = CSV.read("../test/austin-data/coverage_real.csv", DataFrame, header=false)
coverage = convert(Array{Bool, 2}, coverage[:, :])
incidents = CSV.File("../test/austin-data/austin_incidents.csv") |> DataFrame
solverstats = JLD.load("../src/outputs/austin_team_stats_1_03.jld")
amb_deployment = solverstats["amb_deployment"]
model_dict = Dict{String, Symbol}("Stochastic"=>:Stochastic, "Robust01"=>:Robust01, "Robust005"=>:Robust005, "Robust001"=>:Robust001, "Robust0001"=>:Robust0001,
          "Robust00001"=>:Robust00001, "MEXCLP"=>:MEXCLP, "MALP"=>:MALP)


p = DeploymentProblem(
          hourly_calls,
          adjacent_nbhd,
          coverage,
          namb = namb,
          train_filter = (hourly_calls[!,:year] .== 2019) .* (hourly_calls[!,:month] .<= 3)
          )
test_calls = CSV.File("../test/austin-data/austin_test_calls.csv")|> DataFrame
test_calls = test_calls[1:ncalls,:]
println("running $namb ambuances & $ncalls calls")

# distribution of ambulances across all of the stations
x = amb_deployment[model_dict[model_name]][namb]

#establishes the dispatch problem
problem = DispatchProblem(test_calls, hospitals, stations, p.coverage, x, turnaround=turnaround)

#establishes the dispatch model
dispatch = ClosestDispatch(p, problem)

Random.seed!(1234)



running 40 ambuances & 1000 calls


MersenneTwister(UInt32[0x000004d2], Random.DSFMT.DSFMT_state(Int32[-1393240018, 1073611148, 45497681, 1072875908, 436273599, 1073674613, -2043716458, 1073445557, -254908435, 1072827086  …  -599655111, 1073144102, 367655457, 1072985259, -1278750689, 1018350124, -597141475, 249849711, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x000000000000

In [5]:
using AmbulanceDeployment, Dates

Gtk-Message: 12:45:42.457: Failed to load module "canberra-gtk-module"
Gtk-Message: 12:45:42.465: Failed to load module "canberra-gtk-module"


In [12]:
Pkg.add("Query")
using Query

struct EMSEngine{T}
    eventlog::DataFrame
    eventqueue::PriorityQueue{T,Int,Base.Order.ForwardOrdering}
    shortqueue::PriorityQueue{T,Int,Base.Order.ForwardOrdering}
    guiArray::Array{Any, 1}
end

struct gui_event
    #call made / call responded / ambulance arrived / shortfall
    event_type::String
    #neighborhood where call is made from
    neighborhood_id::Int
    # station where call is made from
    deployment_id::Int
    # id of the event generated
    event_id::Int
    ambulance_id::Int
    arrival_time::Int
    remaining_amb::Int
    # current timestamp of event
    timestamp::DateTime
end

## generates the priority queue and instantiates the EMSEngine struct
function generateCalls(problem::DispatchProblem)
    ncalls = nrow(problem.emergency_calls)
    eventlog = DataFrame(
        id = 1:ncalls,
        dispatch_from = zeros(Int, ncalls),
        waittime = fill(0.0, ncalls),
        responsetime = fill(Inf, ncalls),
        scenetime = fill(Inf, ncalls),
        conveytime = fill(Inf, ncalls),
        returntime = fill(Inf, ncalls),
        return_to = zeros(Int, ncalls),
        return_type = fill(:station, ncalls),
        hospital = zeros(Int, ncalls),
        ambulance = zeros(Int, ncalls)
    )
    eventqueue = PriorityQueue{Tuple{Symbol,Int,Int,Int},Int}()
    shortqueue = PriorityQueue{Tuple{Symbol,Int,Int,Int},Int}()
    guiArray = Any[]
    for i in 1:nrow(problem.emergency_calls)
        t = problem.emergency_calls[i, :arrival_seconds]
        enqueue!(eventqueue, (:call, i, t, problem.emergency_calls[i, :neighborhood]), t)
    end
    EMSEngine{Tuple{Symbol,Int,Int,Int}}(eventlog, eventqueue, shortqueue, guiArray) ## returns the ems engine struct
end

function simulate_events!(problem::DispatchProblem, dispatch::ClosestDispatch)
    engine = generateCalls(problem)
    while(~isempty(engine.eventqueue))
        (event, id, t, value) = dequeue!(engine.eventqueue)

        if event == :call
            # if the event is reachable then change event to station2call
            # if not it is a shortfall
            println("calling event id: $id time: $t value: $value")
            call_event!(engine, problem, dispatch, id, t, value)
           
        elseif event == :station2call
            #once the ambulance arrives at the station change event to call2hospital
            println("Ambulance has arrived on the scene for event $id at time $t")
            station2call_event!(engine, problem, dispatch, id, t, value)
        elseif event == :call2hospital
            #once the ambulance arrives at the hospital it will hand over the patient and then 
            #change the deploy back to its station
            println("Ambulance for event $id has arrived at the hospital at time $t")
            call2hosp_event!(engine, problem, dispatch, id, t, value)

        elseif event == :hospital2station
            #once the ambulance drops off the patient it will reroute to the station
            println("Ambulance for event $id is returning to the station at $t")
            return_event!(engine, problem, dispatch, id, t, value)
            
        elseif event == :done
            #increase the number of ambulances at that station now that it is done
            done_event!(engine, problem, dispatch, id, t, value)
            
        end
    end
    println("This is the Short Queue")
    while(~isempty(engine.shortqueue))
       (event, id, t, value) = dequeue!(engine.shortqueue)
        print("$event, $id, $t, $value")
        print("This is not empty")
    end
    engine.eventlog , engine.guiArray
end


function call_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        nbhd::Int; # the neighborhood the call is from
    )

    #check if there is an ambulance within the coverage matrix
    if sum(problem.deployment[problem.coverage[nbhd,:]]) == 0
        @assert false "$id: no ambulance reachable for call at $nbhd"
    #check if one of the ambulances within the coverage is available
    elseif sum(problem.available[problem.coverage[nbhd,:]]) > 0
        i = closest_available(dispatch, id, problem)
        #println("closest available station : $i")
        @assert i > 0 "$id: dispatch from $i to nbhd $nbhd" # assume valid i (enforced by <if> condition)
        update_ambulances!(dispatch, i, -1)
        ems.eventlog[id, :dispatch_from] = i
        @assert problem.available[i] >= 0
        

        travel_time = ceil(Int, 60*2*problem.emergency_calls[id, Symbol("stn$(i)_min")])
        @assert travel_time >= 0
        if(t != problem.emergency_calls[id, :arrival_seconds])
            print("This is a shortfall")
        end
        ems.eventlog[id, :responsetime] = (travel_time + t - problem.emergency_calls[id,:arrival_seconds]) / 60 # minutes

        amb = respond_to!(dispatch, i, t)
        ems.eventlog[id, :ambulance] = amb

        event = gui_event("call responded", problem.emergency_calls[id, :neighborhood], i, id, amb,-1, problem.available[i] ,Dates.now())
        num_ambulances_array = Array{Integer}(undef, size(problem.available, 1))
        for i in 1:size(problem.available, 1)
            num_ambulances_array[i] = problem.available[i]
        end
        push!(ems.guiArray,event)
        push!(ems.guiArray,num_ambulances_array)
        enqueue!(ems.eventqueue, (:station2call, id, t + travel_time, amb), t + travel_time)
    #else queue it
    else
        # println(id, ": call from ", nbhd, " queued behind ", problem.wait_queue[nbhd])
        # event = gui_event("call made", nbhd, -1, id, -1,-1,-1,Dates.now())
        # push!(ems.guiArray,event)
        # problem.shortfalls = problem.shortfalls + 1
        # #push!(problem.shortfalls) # count shortfalls
        # push!(problem.wait_queue[nbhd], id) # queue the emergency call
        event = gui_event("shortfall", problem.emergency_calls[id, :neighborhood], -1, id, -1,-1,-1 ,Dates.now())
        push!(ems.guiArray, event)
        enqueue!(ems.shortqueue, (:call, id, t, problem.emergency_calls[id, :neighborhood]), t)
        println(id, ": call from ", nbhd, " is a shortfall")
    end
end

function station2call_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        amb::Int
    )
    arriveatscene!(dispatch, amb, t)
    # time the ambulance spends at the scene
    scene_time = ceil(Int,60*0.4*rand(problem.turnaround)) # 60sec*0.4*mean(40minutes) ~ 15minutes
    ems.eventlog[id, :scenetime] = scene_time / 60 # minutes
    @assert scene_time > 0
    
    event = gui_event("ambulance arrived", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    enqueue!(ems.eventqueue, (:call2hospital, id, t + scene_time, amb), t + scene_time)
end

function call2hosp_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int, # the id of the emergency call
        t::Int, # the time of the emergency call
        amb::Int
    )
    h = let mintime = Inf, minindex = 0
        for h in 1:nrow(problem.hospitals)
            traveltime = problem.emergency_calls[id, Symbol("hosp$(h)_min")]
            if !isna(traveltime) && !(traveltime=="NA") && traveltime < mintime
                @assert traveltime >= 0
                minindex = h; mintime = traveltime
            end
        end
        minindex
    end
    @assert h != 0
    dispatch.hospital[amb] = ems.eventlog[id, :hospital] = h
    conveytime = 60*15 + ceil(Int, 60*problem.emergency_calls[id, Symbol("hosp$(h)_min")]) # ~20minutes
    ems.eventlog[id, :conveytime] = conveytime / 60 # minutes
    @assert conveytime >= 0 conveytime
    arriveathospital!(dispatch, amb, h, t)
    
    #         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    event = gui_event("ambulance hospital", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    
    enqueue!(ems.eventqueue, (:hospital2station, id, t+conveytime, amb), t+conveytime)
end

function return_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int,
        t::Int,
        amb::Int
    )
    stn = returning_to!(dispatch, amb, t)
    h = dispatch.hospital[amb]
    returntime = ceil(Int,60*2*problem.hospitals[h, Symbol("stn$(stn)_min")]) # ~ 10minutes
    dispatch.hospital[amb] = 0
    ems.eventlog[id, :returntime] = returntime / 60 # minutes
    @assert returntime >= 0 returntime
    t_end = t + returntime
    
#         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    
    event = gui_event("ambulance returning", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    
    enqueue!(ems.eventqueue, (:done, id, t_end, amb), t_end)
    # num_ambulances_array = Array{Integer}(undef, size(problem.available, 1))
    # for i in 1:size(problem.available, 1)
    #     num_ambulances_array[i] = problem.available[i]
    # end
    # push!(ems.guiArray,num_ambulances_array)
end

function done_event!(
        ems::EMSEngine,
        problem::DispatchProblem,
        dispatch::DispatchModel,
        id::Int,
        t::Int,
        amb::Int
    )
    i = dispatch.assignment[amb]
    println("Ambulance from event $id has returned to station $i")
    @assert i > 0 "$id: dispatch from $i to nbhd $nbhd" # assume valid i (enforced by <if> condition)
    update_ambulances!(dispatch, i, 1)
    push!(dispatch.ambulances[i], amb)
    dispatch.status[amb] = :available
    ems.eventlog[id, :return_to] = id
    @assert problem.available[i] > 0
    
#         #call made / call responded / ambulance arrived / shortfall
#     event_type::String
#     #neighborhood where call is made from
#     neighborhood_id::Int
#     # station where call is made from
#     deployment_id::Int
#     # id of the event generated
#     event_id::Int
#     ambulance_id::Int
#     arrival_time::Int
#     remaining_amb::Int
#     # current timestamp of event
#     timestamp::DateTime
    
    event = gui_event("ambulance returned", problem.emergency_calls[id, :neighborhood], -1, id, amb, t,-1 ,Dates.now())
    push!(ems.guiArray, event)
    
    while(~isempty(ems.shortqueue))
        # record time difference
        (event, short_id, short_t, value) = dequeue!(ems.shortqueue)
        enqueue!(ems.eventqueue, (event, short_id, t , value), short_t)
    end
end


  Resolving package versions...
No Changes to `~/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Project.toml`
No Changes to `~/Desktop/UT-Spring2021/SeniorLab/AmbulanceDeployement/AmbulanceDeployment.jl-legacy/Manifest.toml`


done_event! (generic function with 1 method)

In [13]:
@time df, guiArray = simulate_events!(problem, dispatch)

calling event id: 1 time: 38 value: 83
calling event id: 2 time: 575 value: 38
Ambulance has arrived on the scene for event 1 at time 638
calling event id: 3 time: 673 value: 129
calling event id: 4 time: 691 value: 107
calling event id: 5 time: 1166 value: 113
calling event id: 6 time: 1187 value: 51
Ambulance for event 1 has arrived at the hospital at time 1196
calling event id: 7 time: 1450 value: 76
calling event id: 8 time: 1578 value: 67
calling event id: 9 time: 1640 value: 52
Ambulance has arrived on the scene for event 6 at time 1787
calling event id: 10 time: 1838 value: 67
calling event id: 11 time: 1899 value: 67
calling event id: 12 time: 1901 value: 82
Ambulance has arrived on the scene for event 4 at time 1912
Ambulance has arrived on the scene for event 2 at time 1924
calling event id: 13 time: 2091 value: 98
calling event id: 14 time: 2117 value: 113
Ambulance has arrived on the scene for event 3 at time 2140
Ambulance has arrived on the scene for event 8 at time 2178


Ambulance for event 39 has arrived at the hospital at time 12656
Ambulance has arrived on the scene for event 43 at time 12660
calling event id: 45 time: 12678 value: 99
Ambulance has arrived on the scene for event 44 at time 12679
calling event id: 46 time: 12885 value: 124
Ambulance for event 37 has arrived at the hospital at time 13039
Ambulance for event 36 is returning to the station at 13057
calling event id: 47 time: 13075 value: 144
Ambulance has arrived on the scene for event 45 at time 13278
Ambulance for event 35 is returning to the station at 13281
Ambulance for event 43 has arrived at the hospital at time 13329
Ambulance for event 44 has arrived at the hospital at time 13402
Ambulance from event 28 has returned to station 19
Ambulance for event 38 has arrived at the hospital at time 13444
Ambulance for event 41 has arrived at the hospital at time 13454
Ambulance for event 40 has arrived at the hospital at time 13479
Ambulance has arrived on the scene for event 46 at time 1

Ambulance for event 74 has arrived at the hospital at time 24646
calling event id: 78 time: 24714 value: 98
Ambulance from event 68 has returned to station 9
calling event id: 79 time: 25019 value: 98
Ambulance for event 69 is returning to the station at 25082
Ambulance for event 73 is returning to the station at 25102
Ambulance for event 72 is returning to the station at 25141
Ambulance for event 71 is returning to the station at 25272
Ambulance has arrived on the scene for event 76 at time 25277
Ambulance from event 65 has returned to station 3
Ambulance from event 69 has returned to station 8
Ambulance from event 70 has returned to station 15
Ambulance for event 75 has arrived at the hospital at time 25438
Ambulance has arrived on the scene for event 77 at time 25686
Ambulance has arrived on the scene for event 78 at time 25930
calling event id: 80 time: 25961 value: 52
Ambulance from event 72 has returned to station 9
Ambulance for event 74 is returning to the station at 26183
Ambu

Ambulance for event 109 has arrived at the hospital at time 40757
Ambulance for event 107 has arrived at the hospital at time 40785
Ambulance from event 104 has returned to station 5
Ambulance has arrived on the scene for event 110 at time 41073
calling event id: 111 time: 41207 value: 54
Ambulance for event 105 is returning to the station at 41406
Ambulance for event 108 is returning to the station at 41496
Ambulance from event 106 has returned to station 25
calling event id: 112 time: 41779 value: 99
Ambulance has arrived on the scene for event 111 at time 41807
calling event id: 113 time: 42028 value: 128
Ambulance for event 110 has arrived at the hospital at time 42061
Ambulance from event 108 has returned to station 17
calling event id: 114 time: 42207 value: 113
Ambulance for event 109 is returning to the station at 42243
Ambulance from event 105 has returned to station 9
Ambulance for event 107 is returning to the station at 42366
Ambulance has arrived on the scene for event 112

Ambulance for event 142 has arrived at the hospital at time 52512
Ambulance for event 140 has arrived at the hospital at time 52513
Ambulance from event 132 has returned to station 18
Ambulance for event 133 has arrived at the hospital at time 52521
Ambulance has arrived on the scene for event 147 at time 52536
Ambulance for event 145 has arrived at the hospital at time 52568
Ambulance for event 141 has arrived at the hospital at time 52647
Ambulance has arrived on the scene for event 148 at time 52666
Ambulance from event 131 has returned to station 8
Ambulance for event 138 is returning to the station at 52763
Ambulance from event 135 has returned to station 4
calling event id: 153 time: 52887 value: 36
Ambulance has arrived on the scene for event 152 at time 52903
Ambulance has arrived on the scene for event 149 at time 52909
Ambulance for event 143 has arrived at the hospital at time 52952
calling event id: 154 time: 53031 value: 83
Ambulance from event 138 has returned to station 

...

calling event id: 898 time: 252637 value: 129
898: call from 129 is a shortfall
Ambulance for event 878 is returning to the station at 252692
Ambulance from event 876 has returned to station 28
calling event id: 896 time: 252706 value: 52
896: call from 52 is a shortfall
calling event id: 898 time: 252706 value: 129
898: call from 129 is a shortfall
Ambulance for event 886 is returning to the station at 252718
Ambulance has arrived on the scene for event 893 at time 252746
Ambulance from event 880 has returned to station 14
calling event id: 896 time: 252753 value: 52
896: call from 52 is a shortfall
calling event id: 898 time: 252753 value: 129
This is a shortfallAmbulance for event 884 has arrived at the hospital at time 252785
calling event id: 899 time: 252799 value: 68
899: call from 68 is a shortfall
Ambulance from event 869 has returned to station 1
calling event id: 896 time: 252832 value: 52
896: call from 52 is a shortfall
calling event id: 899 time: 252832 value: 68
This is 

Ambulance for event 914 has arrived at the hospital at time 258989
calling event id: 925 time: 259028 value: 142
Ambulance for event 910 is returning to the station at 259042
Ambulance from event 909 has returned to station 22
Ambulance for event 916 is returning to the station at 259074
calling event id: 926 time: 259164 value: 66
Ambulance has arrived on the scene for event 920 at time 259191
Ambulance for event 917 has arrived at the hospital at time 259199
calling event id: 927 time: 259209 value: 51
Ambulance from event 916 has returned to station 8
calling event id: 928 time: 259399 value: 82
Ambulance has arrived on the scene for event 923 at time 259444
Ambulance has arrived on the scene for event 921 at time 259474
calling event id: 929 time: 259529 value: 116
929: call from 116 is a shortfall
Ambulance has arrived on the scene for event 922 at time 259644
Ambulance for event 919 has arrived at the hospital at time 259773
Ambulance from event 897 has returned to station 2
call

calling event id: 952 time: 264000 value: 128
952: call from 128 is a shortfall
calling event id: 953 time: 264027 value: 98
Ambulance has arrived on the scene for event 948 at time 264042
calling event id: 954 time: 264093 value: 141
Ambulance from event 928 has returned to station 8
calling event id: 952 time: 264179 value: 128
952: call from 128 is a shortfall
Ambulance for event 939 has arrived at the hospital at time 264179
Ambulance for event 944 has arrived at the hospital at time 264195
Ambulance for event 933 has arrived at the hospital at time 264196
Ambulance for event 945 has arrived at the hospital at time 264301
Ambulance from event 926 has returned to station 28
calling event id: 952 time: 264424 value: 128
952: call from 128 is a shortfall
Ambulance for event 940 has arrived at the hospital at time 264546
Ambulance has arrived on the scene for event 950 at time 264603
Ambulance for event 947 has arrived at the hospital at time 264629
Ambulance has arrived on the scene f

Ambulance has arrived on the scene for event 972 at time 270214
Ambulance from event 952 has returned to station 17
calling event id: 967 time: 270265 value: 115
967: call from 115 is a shortfall
Ambulance for event 969 has arrived at the hospital at time 270329
Ambulance for event 964 is returning to the station at 270406
calling event id: 974 time: 270537 value: 98
Ambulance for event 968 has arrived at the hospital at time 270716
Ambulance from event 961 has returned to station 26
calling event id: 967 time: 270772 value: 115
967: call from 115 is a shortfall
Ambulance for event 966 is returning to the station at 270913
Ambulance has arrived on the scene for event 970 at time 270921
Ambulance for event 973 has arrived at the hospital at time 270931
Ambulance for event 972 has arrived at the hospital at time 270939
Ambulance has arrived on the scene for event 971 at time 270980
Ambulance for event 965 is returning to the station at 271143
Ambulance from event 955 has returned to stat

(1000×11 DataFrame. Omitted printing of 6 columns
│ Row  │ id    │ dispatch_from │ waittime │ responsetime │ scenetime │
│      │ Int64 │ Int64         │ Float64  │ Float64      │ Float64   │
├──────┼───────┼───────────────┼──────────┼──────────────┼───────────┤
│ 1    │ 1     │ 1             │ 0.0      │ 10.0         │ 9.3       │
│ 2    │ 2     │ 43            │ 0.0      │ 22.4833      │ 20.1333   │
│ 3    │ 3     │ 14            │ 0.0      │ 24.45        │ 18.7333   │
│ 4    │ 4     │ 34            │ 0.0      │ 20.35        │ 11.3167   │
│ 5    │ 5     │ 17            │ 0.0      │ 19.5333      │ 14.5333   │
│ 6    │ 6     │ 22            │ 0.0      │ 10.0         │ 16.7167   │
│ 7    │ 7     │ 39            │ 0.0      │ 49.0833      │ 13.8667   │
│ 8    │ 8     │ 8             │ 0.0      │ 10.0         │ 13.9667   │
│ 9    │ 9     │ 8             │ 0.0      │ 22.7         │ 11.4667   │
│ 10   │ 10    │ 3             │ 0.0      │ 19.2333      │ 17.5      │
⋮
│ 990  │ 990   │ 44      

In [16]:
guiArray

7280-element Array{Any,1}:
 gui_event("call responded", 83, 1, 1, 1, -1, 0, DateTime("2021-03-31T15:19:28.245"))
 Integer[0, 2, 1, 1, 1, 0, 1, 2, 2, 0  …  0, 0, 0, 0, 1, 0, 1, 0, 1, 1]
 gui_event("call responded", 38, 43, 2, 39, -1, 0, DateTime("2021-03-31T15:19:28.245"))
 Integer[0, 2, 1, 1, 1, 0, 1, 2, 2, 0  …  0, 0, 0, 0, 1, 0, 1, 0, 0, 1]
 gui_event("ambulance arrived", 83, -1, 1, 1, 638, -1, DateTime("2021-03-31T15:19:28.246"))
 gui_event("call responded", 129, 14, 3, 13, -1, 0, DateTime("2021-03-31T15:19:28.246"))
 Integer[0, 2, 1, 1, 1, 0, 1, 2, 2, 0  …  0, 0, 0, 0, 1, 0, 1, 0, 0, 1]
 gui_event("call responded", 107, 34, 4, 36, -1, 1, DateTime("2021-03-31T15:19:28.247"))
 Integer[0, 2, 1, 1, 1, 0, 1, 2, 2, 0  …  0, 0, 0, 0, 1, 0, 1, 0, 0, 1]
 gui_event("call responded", 113, 17, 5, 19, -1, 3, DateTime("2021-03-31T15:19:28.247"))
 Integer[0, 2, 1, 1, 1, 0, 1, 2, 2, 0  …  0, 0, 0, 0, 1, 0, 1, 0, 0, 1]
 gui_event("call responded", 51, 22, 6, 24, -1, 1, DateTime("2021-03-31T15:19:28